In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [6]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [7]:
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

In [8]:
ngrams

[(['forty', 'When'], 'winters'),
 (['winters', 'forty'], 'shall'),
 (['shall', 'winters'], 'besiege'),
 (['besiege', 'shall'], 'thy'),
 (['thy', 'besiege'], 'brow,'),
 (['brow,', 'thy'], 'And'),
 (['And', 'brow,'], 'dig'),
 (['dig', 'And'], 'deep'),
 (['deep', 'dig'], 'trenches'),
 (['trenches', 'deep'], 'in'),
 (['in', 'trenches'], 'thy'),
 (['thy', 'in'], "beauty's"),
 (["beauty's", 'thy'], 'field,'),
 (['field,', "beauty's"], 'Thy'),
 (['Thy', 'field,'], "youth's"),
 (["youth's", 'Thy'], 'proud'),
 (['proud', "youth's"], 'livery'),
 (['livery', 'proud'], 'so'),
 (['so', 'livery'], 'gazed'),
 (['gazed', 'so'], 'on'),
 (['on', 'gazed'], 'now,'),
 (['now,', 'on'], 'Will'),
 (['Will', 'now,'], 'be'),
 (['be', 'Will'], 'a'),
 (['a', 'be'], "totter'd"),
 (["totter'd", 'a'], 'weed'),
 (['weed', "totter'd"], 'of'),
 (['of', 'weed'], 'small'),
 (['small', 'of'], 'worth'),
 (['worth', 'small'], 'held:'),
 (['held:', 'worth'], 'Then'),
 (['Then', 'held:'], 'being'),
 (['being', 'Then'], 'asked

In [9]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [10]:
word_to_ix

{'so': 0,
 'made': 1,
 'now,': 2,
 "'This": 3,
 'the': 4,
 'days;': 5,
 'couldst': 6,
 'livery': 7,
 'Will': 8,
 'mine': 9,
 'old,': 10,
 'and': 11,
 'To': 12,
 'Then': 13,
 'fair': 14,
 "beauty's": 15,
 'thou': 16,
 'weed': 17,
 'trenches': 18,
 'his': 19,
 'And': 20,
 'see': 21,
 'thriftless': 22,
 'treasure': 23,
 'shame,': 24,
 'count,': 25,
 'Proving': 26,
 'worth': 27,
 'new': 28,
 'when': 29,
 "totter'd": 30,
 'praise': 31,
 'held:': 32,
 'deep': 33,
 'brow,': 34,
 'proud': 35,
 'How': 36,
 'asked,': 37,
 'of': 38,
 'within': 39,
 'an': 40,
 'small': 41,
 "feel'st": 42,
 'When': 43,
 'thy': 44,
 'succession': 45,
 'own': 46,
 'cold.': 47,
 'to': 48,
 'thine!': 49,
 'field,': 50,
 'on': 51,
 'lies,': 52,
 'child': 53,
 'be': 54,
 'old': 55,
 'all-eating': 56,
 'make': 57,
 'all': 58,
 'sunken': 59,
 'praise.': 60,
 'much': 61,
 'gazed': 62,
 'my': 63,
 'dig': 64,
 'warm': 65,
 'Where': 66,
 'sum': 67,
 'Were': 68,
 'besiege': 69,
 'by': 70,
 'Shall': 71,
 'being': 72,
 "deserv'd"

In [11]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [12]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [13]:
for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)

In [14]:
print(losses)  # The loss decreased every iteration over the training data!

[518.0279240608215, 515.7042031288147, 513.3921675682068, 511.0924482345581, 508.80477476119995, 506.5275626182556, 504.2590973377228, 501.99819803237915, 499.74469804763794, 497.49769020080566]


In [15]:
# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

tensor([-0.9105, -0.1426,  0.3001,  1.4573,  0.2307, -0.1474, -0.6362,  0.3518,
        -0.0297, -0.5678], grad_fn=<SelectBackward>)
